In [1]:
import numpy as np
import pandas as pd
import io
import re
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt
from gensim.models import Word2Vec


In [2]:
data = pd.read_csv('/Users/minhyuk/study/example/korean_drinks_food.csv')
data.head()


,전통주명,가격,용량,주종,도수,단맛,신맛,바디감,청량감,잘 어울리는 음식,주원료,제조사,유형
0,구름을 벗삼아,"1,500",750ml,탁주,6.0,3,3,3,3,가리비,"정제수, 찹쌀·백미(국내산햅쌀 100%), 입국",NaN,4
1,해창 생막걸리 6도,"3,000",900ml,탁주,6.0,1,1,2,3,간재미 찜,"햅쌀, 찹쌀, 물, 누룩",해창주조장,2
2,도문대작 생막걸리,"2,000",750ml,증류주,10.0,1,3,2,3,갈비찜,"강릉 쌀, 누룩, 갯방풍나물, 정제수",방풍도가,3
3,이동 생 쌀 막걸리,"1,300",750ml,탁주,6.0,1,3,2,3,갈비찜,"정제수, 백미, 팽화미, 입국, 아스파탐 등",이동주조,3
4,동정춘,"18,000",500ml,탁주,8.0,4,3,3,2,"갈비찜, 단맛이 약간 있는 음식","쌀, 누룩, 정제수",농업회사법인㈜산수,4


In [3]:
shop_list = data[['전통주명', '잘 어울리는 음식']]
shop_list.head()


,전통주명,잘 어울리는 음식
0,구름을 벗삼아,가리비
1,해창 생막걸리 6도,간재미 찜
2,도문대작 생막걸리,갈비찜
3,이동 생 쌀 막걸리,갈비찜
4,동정춘,"갈비찜, 단맛이 약간 있는 음식"


In [4]:
df = pd.DataFrame(shop_list, columns=['전통주명', '잘 어울리는 음식'])
df.head()


,전통주명,잘 어울리는 음식
0,구름을 벗삼아,가리비
1,해창 생막걸리 6도,간재미 찜
2,도문대작 생막걸리,갈비찜
3,이동 생 쌀 막걸리,갈비찜
4,동정춘,"갈비찜, 단맛이 약간 있는 음식"


In [5]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()


In [6]:
from tqdm import tqdm


noun_list = []
for content in tqdm(df['잘 어울리는 음식']):
    nouns = okt.nouns(content)
    noun_list.append(nouns)

df['nouns'] = noun_list
df.head()


100%|██████████| 295/295 [00:02<00:00, 130.70it/s]


,전통주명,잘 어울리는 음식,nouns
0,구름을 벗삼아,가리비,[가리비]
1,해창 생막걸리 6도,간재미 찜,"[간, 재미, 찜]"
2,도문대작 생막걸리,갈비찜,[갈비찜]
3,이동 생 쌀 막걸리,갈비찜,[갈비찜]
4,동정춘,"갈비찜, 단맛이 약간 있는 음식","[갈비찜, 단맛, 약간, 음식]"


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()


In [8]:
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector)
model = DBSCAN(eps=0.4, min_samples=5, metric="cosine")
result = model.fit_predict(vector)

df['result'] = result
df.head()


,전통주명,잘 어울리는 음식,nouns,result
0,구름을 벗삼아,가리비,[가리비],-1
1,해창 생막걸리 6도,간재미 찜,"[간, 재미, 찜]",-1
2,도문대작 생막걸리,갈비찜,[갈비찜],0
3,이동 생 쌀 막걸리,갈비찜,[갈비찜],0
4,동정춘,"갈비찜, 단맛이 약간 있는 음식","[갈비찜, 단맛, 약간, 음식]",0


In [9]:
for cluster_num in set(result):
    if (cluster_num == -1 or cluster_num == 0):
        continue
    else:
        print("cluster num : {}".format(cluster_num))
        temp_df = df[df['result'] == cluster_num]
        for menu in temp_df['잘 어울리는 음식']:
            print(menu)
        print()


cluster num : 1
곱창 전골, 연포탕
두부전골
두부전골
버섯 전골
버섯 전골류
버섯요리, 송이버섯 전골
버섯전골

cluster num : 2
과일 샐러드
샐러드
샐러드
샐러드
샐러드
샐러드
샐러드
샐러드
샐러드, 두부
샐러드, 카르파쵸, 가벼운 튀김류
샐러드, 해산물, 케이크, 마카롱
애플파이, 과실 샐러드
야채 샐러드
야채 샐러드
야채 샐러드

cluster num : 3
맥주에 어울리는 모든 안주
모짜렐라 치즈, 카망베르, 스모키 치즈
모짜렐라치즈
안주 없이
안주 없이 즐김
육류, 치즈
육회 안주
치즈, 과일 안주

cluster num : 4
불고기, 양념갈비
양념 갈비
양념갈비
양념갈비
양념갈비

cluster num : 5
스테이크
스테이크
스테이크
양꼬치, 항정살, 스테이크
함박 스테이크, 파스타

cluster num : 6
장어구이
장어구이
장어구이
장어구이
장어구이

cluster num : 7
홍어, 오징어
홍어무침
홍어무침
홍어무침
홍어무침, 오징어무침

